In [12]:
import numpy as np
import os
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh, ArpackNoConvergence

import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.layers import Layer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets
from tensorflow.keras.regularizers import l2
from tensorflow.keras import activations, initializers, constraints
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

In [14]:
node_data_path = 'cora/cora.content'
edge_data_path = 'cora/cora.cites'

# 数据集链接：https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
idx_features_labels = np.genfromtxt(node_data_path, dtype=np.dtype(str))

In [13]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
labels = encode_onehot(idx_features_labels[:, -1])

In [15]:
# build graph
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}

edges_unordered = np.genfromtxt(edge_data_path, dtype=np.int32)

edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                 dtype=np.int32).reshape(edges_unordered.shape)

adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

# build symmetric adjacency matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

print('Dataset(adj) has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))

Dataset(adj) has 2708 nodes, 5429 edges, 1433 features.


In [17]:
x = features.todense()
y = labels

In [18]:
batch_size = adj.shape[1]

In [19]:
batch_size

2708

In [20]:
x /= x.sum(1).reshape(-1, 1)

In [22]:
def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def get_splits(y):
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    train_mask = sample_mask(idx_train, y.shape[0])
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask

y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

C:\Users\18438\AppData\Local\Temp/ipykernel_17392/2324728083.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(mask, dtype=np.bool)


In [48]:
class Config(object):
    filter = 'localpool'    # Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016)
    # filter = 'chebyshev'  # Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)
    max_degree = 2  # maximum polynomial degree
    sym_norm = True  # symmetric (True) vs. left-only (False) normalization
    NB_EPOCH = 20
    PATIENCE = 10  # early stopping patience
    support = 1
    epochs = 5

In [28]:
def preprocess_adj(adj, symmetric=True):
    adj = adj + sp.eye(adj.shape[0])
    adj = normalize_adj(adj, symmetric)
    return adj

def normalize_adj(adj, symmetric=True):
    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d).tocsr()
    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj).tocsr()
    return a_norm

def get_inputs(adj, x):
    if Config.filter == 'localpool':
        print('Using local pooling filters...')
        adj_ = preprocess_adj(adj, Config.sym_norm)
        adj_ = adj_.todense()
        graph = [x, adj_]
        adj_input = [Input(batch_shape=(None, None), sparse=False, name='adj_input')]
    elif Config.filter == 'chebyshev':
        print('Using Chebyshev polynomial basis filters...')
        L = normalized_laplacian(adj, Config.sym_norm)
        L_scaled = rescale_laplacian(L)
        T_k = chebyshev_polynomial(L_scaled, Config.max_degree)
        support = Config.max_degree + 1
        graph = [x] + T_k
        adj_input = [Input(batch_shape=(None, None), sparse=False, name='adj_input') for _ in range(support)]
    else:
        raise Exception('Invalid filter type.')
    return graph, adj_input

x_graph, adj_input = get_inputs(adj, x)

Using local pooling filters...


In [35]:
class GraphConvolution(Layer):
    """Basic graph convolution layer as in https://arxiv.org/abs/1609.02907"""
    def __init__(self, units, support=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)

        super(GraphConvolution, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.supports_masking = True
        self.support = support
        assert support >= 1.0

    def compute_output_shape(self, input_shapes):
        features_shape = input_shapes[0]
        output_shape = (features_shape[0], self.units)
        return output_shape  # (batch_size, output_dim)

    def build(self, input_shapes):
        features_shape = input_shapes[0]
        assert len(features_shape) == 2
        input_dim = features_shape[1]
        self.kernel = self.add_weight(shape=(input_dim * self.support,
                                             self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.built = True
    
    # core code
    def call(self, inputs, mask=None):
        features = inputs[0]
        basis = inputs[1:] # this is a list
        supports = list()
        for i in range(self.support):
            # A * X
            supports.append(K.dot(basis[i], features))
        supports = K.concatenate(supports, axis=1)
        # A * X * W
        output = K.dot(supports, self.kernel)
        if tf.is_tensor(self.bias) :
            output += self.bias
        return self.activation(output)

    def get_config(self):
        config = {'units': self.units,
                  'support': self.support,
                  'activation': activations.serialize(self.activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(
                      self.kernel_initializer),
                  'bias_initializer': initializers.serialize(
                      self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(
                      self.kernel_regularizer),
                  'bias_regularizer': regularizers.serialize(
                      self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(
                      self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(
                      self.kernel_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(GraphConvolution, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [42]:
def build_model(x, y, adj_input):
    fea_input = Input(batch_shape=(None, x.shape[1]), name='fea_input')
    net = Dropout(0.2)(fea_input)
    net = GraphConvolution(128, Config.support, activation='relu', kernel_regularizer=l2(5e-4))([net] + adj_input)
    net = Dropout(0.2)(net)
    net = GraphConvolution(64, Config.support, activation='relu', kernel_regularizer=l2(5e-4))([net] + adj_input)
    net = Dropout(0.2)(net)
    net = Flatten()(net)
    output = Dense(y.shape[1], activation='softmax')(net)
    # output = GraphConvolution(y.shape[1], Config.support, activation='softmax')([net] + adj_input)
    model = Model(inputs=[fea_input] + adj_input, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
    return model

In [43]:
model = build_model(x, y, adj_input)

In [46]:
def accuracy(preds, labels):
    return np.mean(np.equal(np.argmax(labels, 1), np.argmax(preds, 1)))


def evaluate_preds(preds, labels, indices):
    split_loss = list()
    split_acc = list()
    for y_split, idx_split in zip(labels, indices):
        split_loss.append(categorical_crossentropy(preds[idx_split], y_split[idx_split]))
        split_acc.append(accuracy(preds[idx_split], y_split[idx_split]))
    return split_loss, split_acc


def categorical_crossentropy(preds, labels):
    return np.mean(-np.log(np.extract(labels, preds)))


def train_model(x, y, model, train_mask, y_train, y_val, idx_train, idx_val, batch_size):
    for i in range(Config.epochs):
        model.fit(x, y, sample_weight=train_mask, batch_size=batch_size, epochs=1, shuffle=False, verbose=1)
        y_pred = model.predict(x, batch_size=batch_size)
        train_val_loss, train_val_acc = evaluate_preds(y_pred, [y_train, y_val], [idx_train, idx_val])
        print("train_loss= {:.2f}".format(train_val_loss[0]), "train_acc= {:.2f}".format(train_val_acc[0]),
              "val_loss= {:.2f}".format(train_val_loss[1]), "val_acc= {:.2f}".format(train_val_acc[1]))
    return model


def estimate_model(model, x, y_test, idx_test, batch_size):
    y_pred = model.predict(x, batch_size=batch_size)
    test_loss, test_acc = evaluate_preds(y_pred, [y_test], [idx_test])
    print("Test set results:", "loss= {:.2f}".format(test_loss[0]), "accuracy= {:.4f}".format(test_acc[0]))

In [49]:
model = train_model(x_graph, y, model, train_mask, y_train, y_val, idx_train, idx_val, batch_size)

1/1 [==============================] - 0s 996us/step - loss: 0.0969
train_loss= 1.80 train_acc= 0.31 val_loss= 1.80 val_acc= 0.35
1/1 [==============================] - 0s 2ms/step - loss: 0.0965
train_loss= 1.79 train_acc= 0.31 val_loss= 1.80 val_acc= 0.35
1/1 [==============================] - 0s 996us/step - loss: 0.0965
train_loss= 1.79 train_acc= 0.29 val_loss= 1.80 val_acc= 0.34
1/1 [==============================] - 0s 999us/step - loss: 0.0976
train_loss= 1.79 train_acc= 0.31 val_loss= 1.80 val_acc= 0.35
1/1 [==============================] - 0s 996us/step - loss: 0.0976
train_loss= 1.79 train_acc= 0.31 val_loss= 1.80 val_acc= 0.34
